In [2]:
import nibabel as nib 
import numpy as np 
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline 

## Functional Masks (ROIs)

For our univariate analyses, we defined functional ROIs based on the sensorimotor representations of the left/right 
hand and feet for each participant. We used condition-specific contrasts from the motor localiser scans. The relevant (right/left hand vs lips, right/left foot vs lips) low-level contrasts were first averaged across both (pre and post) scans. 

To create left/right hand ROIs, for each participant, the 200 most active voxels were selected from the averaged left/right hand vs lips contrast. For the individual feet ROIs, a similar procedure was employed, selecting the 100 most active voxels from the averaged left/right foot vs lips contrast. Voxel selection was restricted to the primary somatosensory (S1) and motor (M1) cortices as derived from the maximum probabilistic maps (thresholded at 25%) of the Juelich Histological Atlas. Voxels from both feet ROIs were combined into a single region of interest used in the subsequent analyses.

In [3]:
augmented = ['SF1', 'SF2', 'SF3', 'SF4', 'SF5', 'SF6', 'SF7', 'SF8', 'SF11', 'SF12', 'SF13', 'SF14',
          'SF15', 'SF16', 'SF17', 'SF19', 'SF21', 'SF22', 'SF23', 'SF24']
controls = ['CF1', 'CF2', 'CF4', 'CF5', 'CF6', 'CF7', 'CF8', 'CF9', 'CF10', 'CF11', 'CF12']
sessions = ['pre', 'post']

### 1. Average pre- and post- z-stats

* LH: zstat6 (left hand vs lips)
* RH: zstat7 (right hand vs lips)
* LF: zstat8 (left foot vs lips)
* RF: zstat9 (right foot vs lips)

In [3]:
for subj in augmented + controls:
        !fslmaths /vols/Data/soma/6Finger/$subj/pre/model/bodyloc.feat/stats/zstat6.nii.gz -add /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/stats/zstat6.nii.gz -div 2 /vols/Data/soma/6Finger/$subj/coreg/zstat6_avg.nii.gz
        !fslmaths /vols/Data/soma/6Finger/$subj/pre/model/bodyloc.feat/stats/zstat7.nii.gz -add /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/stats/zstat7.nii.gz -div 2 /vols/Data/soma/6Finger/$subj/coreg/zstat7_avg.nii.gz
        !fslmaths /vols/Data/soma/6Finger/$subj/pre/model/bodyloc.feat/stats/zstat8.nii.gz -add /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/stats/zstat8.nii.gz -div 2 /vols/Data/soma/6Finger/$subj/coreg/zstat8_avg.nii.gz
        !fslmaths /vols/Data/soma/6Finger/$subj/pre/model/bodyloc.feat/stats/zstat9.nii.gz -add /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/stats/zstat9.nii.gz -div 2 /vols/Data/soma/6Finger/$subj/coreg/zstat9_avg.nii.gz

### 2. Register the Juelich_M1S1 mask to func space (bodyloc)

In [4]:
for subj in augmented + controls:
    !fsl_sub -q veryshort.q invwarp -w /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/reg/highres2standard_warp \
    -o /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/reg/standard2highres_warp -r /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/reg/highres

63947
63948
63949
63950
63951
63952
63953
63954
63955
63956
63957


In [7]:
for subj in augmented + controls:
    !fsl_sub -q veryshort.q applywarp -i /vols/Data/soma/6Finger/masks/vol/Juelich_M1S1_dil.nii.gz \
    -r /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/example_func \
    -o /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz -w /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/reg/standard2highres_warp.nii.gz \
    --postmat=/vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/reg/highres2example_func.mat

64234
64235
64236
64237
64238
64239
64240
64241
64242
64243
64244


In [8]:
for subj in augmented + controls:
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz -thr 0.2 -bin /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz    

### 2. Create masks based on the average z-stats

Mask the zstat_avg images with the Juelich_M1S1 mask, then choose 200 strongest voxels for the hand ROIs and 100 strongest voxels from each foot for the feet ROI

In [9]:
for subj in augmented + controls: 
    
    if subj=='SF12':
        !fslmaths /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/stats/zstat6.nii.gz -mas /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz /vols/Data/soma/6Finger/$subj/coreg/zstat6_avg_masked.nii.gz 
        !fslmaths /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/stats/zstat7.nii.gz -mas /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz /vols/Data/soma/6Finger/$subj/coreg/zstat7_avg_masked.nii.gz 
        !fslmaths /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/stats/zstat8.nii.gz -mas /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz /vols/Data/soma/6Finger/$subj/coreg/zstat8_avg_masked.nii.gz 
        !fslmaths /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/stats/zstat9.nii.gz -mas /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz /vols/Data/soma/6Finger/$subj/coreg/zstat9_avg_masked.nii.gz         
    else:
        !fslmaths /vols/Data/soma/6Finger/$subj/coreg/zstat6_avg.nii.gz -mas /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz /vols/Data/soma/6Finger/$subj/coreg/zstat6_avg_masked.nii.gz 
        !fslmaths /vols/Data/soma/6Finger/$subj/coreg/zstat7_avg.nii.gz -mas /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz /vols/Data/soma/6Finger/$subj/coreg/zstat7_avg_masked.nii.gz 
        !fslmaths /vols/Data/soma/6Finger/$subj/coreg/zstat8_avg.nii.gz -mas /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz /vols/Data/soma/6Finger/$subj/coreg/zstat8_avg_masked.nii.gz 
        !fslmaths /vols/Data/soma/6Finger/$subj/coreg/zstat9_avg.nii.gz -mas /vols/Data/soma/6Finger/$subj/masks/Juelich_M1S1.nii.gz /vols/Data/soma/6Finger/$subj/coreg/zstat9_avg_masked.nii.gz 

    nii_LH=nib.load('../{}/coreg/zstat6_avg_masked.nii.gz'.format(subj))
    nii_RH=nib.load('../{}/coreg/zstat7_avg_masked.nii.gz'.format(subj))
    nii_LF=nib.load('../{}/coreg/zstat8_avg_masked.nii.gz'.format(subj))
    nii_RF=nib.load('../{}/coreg/zstat9_avg_masked.nii.gz'.format(subj))
    
    zstat_LH = nii_LH.get_data()
    zstat_RH = nii_RH.get_data()
    zstat_LF = nii_LF.get_data()
    zstat_RF = nii_RF.get_data()
    
    zstat_list_LH = zstat_LH[zstat_LH>0]
    zstat_list_LH[::-1].sort()
    zstat_list_RH = zstat_RH[zstat_RH>0]
    zstat_list_RH[::-1].sort()
    zstat_list_LF = zstat_LF[zstat_LF>0]
    zstat_list_LF[::-1].sort()
    zstat_list_RF = zstat_RF[zstat_RF>0]
    zstat_list_RF[::-1].sort()

    thr_LH = zstat_list_LH[200]
    thr_RH = zstat_list_RH[200]
    thr_LF = zstat_list_LF[100]
    thr_RF = zstat_list_RF[100]
        
    mask_LH = zstat_LH > thr_LH
    mask_RH = zstat_RH > thr_RH
    mask_LF = zstat_LF > thr_LF
    mask_RF = zstat_RF > thr_RF

    nib.save(nib.Nifti1Image(mask_LH,nii_LH.affine,header=nii_LH.header), '/vols/Data/soma/6Finger/' + subj + '/masks/LH200mask.nii.gz')
    nib.save(nib.Nifti1Image(mask_RH,nii_RH.affine,header=nii_RH.header), '/vols/Data/soma/6Finger/' + subj + '/masks/RH200mask.nii.gz')
    nib.save(nib.Nifti1Image(mask_LF,nii_LF.affine,header=nii_LF.header), '/vols/Data/soma/6Finger/' + subj + '/masks/LF100mask.nii.gz')
    nib.save(nib.Nifti1Image(mask_RF,nii_RF.affine,header=nii_RF.header), '/vols/Data/soma/6Finger/' + subj + '/masks/RF100mask.nii.gz')

    !fslmaths /vols/Data/soma/6Finger/$subj/masks/LH200mask.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/LH200mask.nii.gz
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/RH200mask.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/RH200mask.nii.gz
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/LF100mask.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/LF100mask.nii.gz
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/RF100mask.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/RF100mask.nii.gz
     
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/LF100mask.nii.gz -add /vols/Data/soma/6Finger/$subj/masks/RF100mask.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/BF200mask.nii.gz
    
    print(subj + ' done')

<ipython-input-9-482f02b4c2be>:20: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  zstat_LH = nii_LH.get_data()
<ipython-input-9-482f02b4c2be>:21: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  zstat_RH = nii_RH.get_data()
<ipython-input-9-482f02b4c2be>:22: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from vers

CF1 done
CF2 done
CF4 done
CF5 done
CF6 done
CF7 done
CF8 done
CF9 done
CF10 done
CF11 done
CF12 done


In [10]:
for subj in augmented + controls:
    
    !flirt -in /vols/Data/soma/6Finger/$subj/masks/LH200mask.nii.gz -ref /vols/Data/soma/6Finger/$subj/coreg/anatMidspace.nii.gz -applyxfm -init /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/reg/example_func2highres.mat -out /vols/Data/soma/6Finger/$subj/masks/LH200mask_highres.nii.gz
    !flirt -in /vols/Data/soma/6Finger/$subj/masks/RH200mask.nii.gz -ref /vols/Data/soma/6Finger/$subj/coreg/anatMidspace.nii.gz -applyxfm -init /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/reg/example_func2highres.mat -out /vols/Data/soma/6Finger/$subj/masks/RH200mask_highres.nii.gz
   
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/LH200mask_highres.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/LH200mask_highres.nii.gz
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/RH200mask_highres.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/RH200mask_highres.nii.gz
    

### 3. Register the masks to restMidspace

In [43]:
for subj in augmented + controls:
    
    if subj=='SF12':
        ! flirt -in /vols/Data/soma/6Finger/$subj/post/model/bodyloc.feat/example_func.nii.gz -ref /vols/Data/soma/6Finger/$subj/coreg/restMidspace.nii.gz -omat /vols/Data/soma/6Finger/$subj/coreg/bodylocMidspace2restMidspace.mat
    else:
        ! flirt -in /vols/Data/soma/6Finger/$subj/coreg/bodylocMidspace.nii.gz -ref /vols/Data/soma/6Finger/$subj/coreg/restMidspace.nii.gz -omat /vols/Data/soma/6Finger/$subj/coreg/bodylocMidspace2restMidspace.mat
    
    !flirt -in /vols/Data/soma/6Finger/$subj/masks/LH200mask.nii.gz -ref /vols/Data/soma/6Finger/$subj/coreg/restMidspace.nii.gz -applyxfm -init /vols/Data/soma/6Finger/$subj/coreg/bodylocMidspace2restMidspace.mat -out /vols/Data/soma/6Finger/$subj/masks/LH200mask_rest.nii.gz
    !flirt -in /vols/Data/soma/6Finger/$subj/masks/RH200mask.nii.gz -ref /vols/Data/soma/6Finger/$subj/coreg/restMidspace.nii.gz -applyxfm -init /vols/Data/soma/6Finger/$subj/coreg/bodylocMidspace2restMidspace.mat -out /vols/Data/soma/6Finger/$subj/masks/RH200mask_rest.nii.gz
    !flirt -in /vols/Data/soma/6Finger/$subj/masks/LF100mask.nii.gz -ref /vols/Data/soma/6Finger/$subj/coreg/restMidspace.nii.gz -applyxfm -init /vols/Data/soma/6Finger/$subj/coreg/bodylocMidspace2restMidspace.mat -out /vols/Data/soma/6Finger/$subj/masks/LF100mask_rest.nii.gz
    !flirt -in /vols/Data/soma/6Finger/$subj/masks/RF100mask.nii.gz -ref /vols/Data/soma/6Finger/$subj/coreg/restMidspace.nii.gz -applyxfm -init /vols/Data/soma/6Finger/$subj/coreg/bodylocMidspace2restMidspace.mat -out /vols/Data/soma/6Finger/$subj/masks/RF100mask_rest.nii.gz
    !flirt -in /vols/Data/soma/6Finger/$subj/masks/BF200mask.nii.gz -ref /vols/Data/soma/6Finger/$subj/coreg/restMidspace.nii.gz -applyxfm -init /vols/Data/soma/6Finger/$subj/coreg/bodylocMidspace2restMidspace.mat -out /vols/Data/soma/6Finger/$subj/masks/BF200mask_rest.nii.gz
   
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/LH200mask_rest.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/LH200mask_rest.nii.gz
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/RH200mask_rest.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/RH200mask_rest.nii.gz
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/LF100mask_rest.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/LF100mask_rest.nii.gz
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/RF100mask_rest.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/RF100mask_rest.nii.gz
    !fslmaths /vols/Data/soma/6Finger/$subj/masks/BF200mask_rest.nii.gz -bin /vols/Data/soma/6Finger/$subj/masks/BF200mask_rest.nii.gz


### 4. Extract mean timecourses

In [45]:
for subj in augmented + controls:
    for ses in sessions:
        ! fslmeants -i /vols/Data/soma/6Finger/$subj/$ses/model/rest.feat/stats/res4d.nii.gz -m /vols/Data/soma/6Finger/$subj/masks/LH200mask_rest.nii.gz -o /vols/Data/soma/6Finger/$subj/$ses/masks/LH_seed.txt
        ! fslmeants -i /vols/Data/soma/6Finger/$subj/$ses/model/rest.feat/stats/res4d.nii.gz -m /vols/Data/soma/6Finger/$subj/masks/RH200mask_rest.nii.gz -o /vols/Data/soma/6Finger/$subj/$ses/masks/RH_seed.txt
        ! fslmeants -i /vols/Data/soma/6Finger/$subj/$ses/model/rest.feat/stats/res4d.nii.gz -m /vols/Data/soma/6Finger/$subj/masks/LF100mask_rest.nii.gz -o /vols/Data/soma/6Finger/$subj/$ses/masks/LF_seed.txt
        ! fslmeants -i /vols/Data/soma/6Finger/$subj/$ses/model/rest.feat/stats/res4d.nii.gz -m /vols/Data/soma/6Finger/$subj/masks/RF100mask_rest.nii.gz -o /vols/Data/soma/6Finger/$subj/$ses/masks/RF_seed.txt
        ! fslmeants -i /vols/Data/soma/6Finger/$subj/$ses/model/rest.feat/stats/res4d.nii.gz -m /vols/Data/soma/6Finger/$subj/masks/BF200mask_rest.nii.gz -o /vols/Data/soma/6Finger/$subj/$ses/masks/BF_seed.txt
    print(subj + ' done')

SF1 done
SF2 done
SF3 done
SF4 done
SF5 done
SF6 done
SF7 done
SF8 done
SF11 done
SF12 done
SF13 done
SF14 done
SF15 done
SF16 done
SF17 done
SF19 done
SF21 done
SF22 done
SF23 done
SF24 done
